In [1]:
import numpy as np
import pandas as pd
import spacy
from spacy import displacy
from nltk.tokenize import sent_tokenize

In [2]:
import fr_core_news_md

nlp = fr_core_news_md.load()

In [3]:
def seg2sent(seg):
    phrase = ""
    for token in seg:
        phrase += token.text + " " 
    return phrase

In [22]:
def make_list_spo(doc):
    # taking a spacy doc object, return list of source-predicat-opinion
    ccomps = []
    for token in doc:
        if token.dep_ == 'ccomp':
            ccomps.append(token)
    list_spo = []
    for ccomp in ccomps:
        spo = {}
        #opinion
        spo['opinion'] = [tok for tok in ccomp.subtree]
        #predicat
        predicat = []
        for token in ccomp.head.lefts:
            if token.dep_ in ['prt','aux','aux:tense','neg','advmod']: 
                predicat.append(token)
        predicat.append(ccomp.head)   #ccomp.head is the principle predicat
        for token in ccomp.head.rights:
            if token.dep_ in ['prt','aux','aux:tense','neg','advmod']: 
                predicat.append(token)        
        spo['predicat'] = predicat
        #source
        source = []
        if ccomp.head.dep_ == 'conj':
            for pred in predicat:
                source += [x for tok in pred.head.children if tok.dep_ in ['nsubj'] for x in tok.subtree]            
        else:
            for pred in predicat:
                source += [x for tok in pred.children if tok.dep_ in ['nsubj'] for x in tok.subtree]
        spo['source'] = source
        list_spo.append(spo)
        
    if len(list_spo) == 0:
        print("No opinion found !")
    
    return list_spo

In [17]:
df = pd.read_csv("/Users/rubing/Documents/Donnees/covax_content_sample_2000_medium_sized.csv").drop("Unnamed: 0", axis=1)

In [18]:
TEXT=df.extracted_text[0]
print(TEXT)

Un million de tests pourraient être réalisés d'ici la fin de la semaine. Et un traitement pourrait être disponible d'ici l'été ou l'automne, selon Mike Pence.
Aux États-Unis aussi, les morts se multiplient. Avec quatre nouveaux décès enregistrés lundi, six personnes ont succombé au coronavirus outre-Atlantique, et la barre des cent contaminés a été atteinte. Selon le New York Times , l'administration Trump a alors annoncé qu' un million de tests pourraient être réalisés d'ici la fin de la semaine. Des compagnies privées et des laboratoires académiques ont été poussés à développer et valider leur propre dépistage.  
LIRE AUSSI >> Coronavirus : les Américains sans couverture santé dans l'impasse  
D'autre part, un traitement médical contre le nouveau coronavirus pourrait être disponible "d'ici l'été ou le début de l'automne", a affirmé lundi le vice-président américain Mike Pence , qui coordonne la lutte contre l'épidémie aux États-Unis.  Offre limitée. 2 mois pour 1€ sans engagement Je 

Ex: Text 4, 5

C-Ex: Text 0, sent 9, 11

In [19]:
DOC = nlp(TEXT)
sents = list(DOC.sents)

In [30]:
i = 

sent = sents[i]
print(sent)

D'autre part, un traitement médical contre le nouveau coronavirus pourrait être disponible "d'ici l'été ou le début de l'automne", a affirmé lundi le vice-président américain Mike Pence , qui coordonne la lutte contre l'épidémie aux États-Unis.  


In [33]:
sent = nlp(""""Les traitements thérapeutiques pour apporter un réconfort aux gens qui contractent le coronavirus pourraient être disponibles d'ici l'été ou le début de l'automne", a-t-il affirmé lors d'un point presse, ajoutant qu'un vaccin contre la maladie "pourrait ne pas être disponible avant la fin de l'année ou le début de l'année prochaine". """)

In [34]:
print(sent)
print('\n')


list_spo = make_list_spo(sent)

for i, spo in enumerate(list_spo):
    print(f"Source: {seg2sent(spo['source'])}")
    print(f"Predicat: {seg2sent(spo['predicat'])}")
    print(f"Opinion: {seg2sent(spo['opinion'])}")
    print("\n")

"Les traitements thérapeutiques pour apporter un réconfort aux gens qui contractent le coronavirus pourraient être disponibles d'ici l'été ou le début de l'automne", a-t-il affirmé lors d'un point presse, ajoutant qu'un vaccin contre la maladie "pourrait ne pas être disponible avant la fin de l'année ou le début de l'année prochaine". 


Source: 
Predicat: ajoutant 
Opinion: qu' un vaccin contre la maladie " pourrait ne pas être disponible avant la fin de l' année ou le début de l' année prochaine " 




In [35]:
displacy.render(sent,style='dep',options={'compact':True})

In [8]:
DOC = nlp(TEXT)
for i,sent in enumerate(DOC.sents):
    print('\n')
    print(str(i)+": "+sent.text)
    print('\n')
    list_spo = make_list_spo(sent)
    for i, spo in enumerate(list_spo):
        print(f"Source: {seg2sent(spo['source'])}")
        print(f"Predicat: {seg2sent(spo['predicat'])}")
        print(f"Opinion: {seg2sent(spo['opinion'])}")
        print("\n")



0: Un million de tests pourraient être réalisés d'ici la fin de la semaine.


No opinion found !


1: Et un traitement pourrait être disponible d'ici l'été ou l'automne, selon Mike Pence.



No opinion found !


2: Aux États-Unis aussi, les morts se multiplient.


No opinion found !


3: Avec quatre nouveaux décès enregistrés lundi, six personnes ont succombé au coronavirus outre-Atlantique, et la barre des cent contaminés a été atteinte.


No opinion found !


4: Selon le New York Times , l'administration Trump a alors annoncé qu' un million de tests pourraient être réalisés d'ici la fin de la semaine.


Source: l' administration Trump 
Predicat: a alors annoncé 
Opinion: qu ' un million de tests pourraient être réalisés d' ici la fin de la semaine 




5: Des compagnies privées et des laboratoires académiques ont été poussés à développer et valider leur propre dépistage.  



No opinion found !


6: LIRE AUSSI


No opinion found !


7: >>


No opinion found !


8: Coronavirus : les